[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/redun-lamin-fasta/blob/main/docs/redun.ipynb)

# Redun

Here, we'll see how to track redun workflow runs with LaminDB.

```{note}

This use case is based on [github.com/ricomnl/bioinformatics-pipeline-tutorial](https://github.com/ricomnl/bioinformatics-pipeline-tutorial/tree/redun).

```

## Setup

In [ ]:
!lamin init --storage .  --name redun-lamin-fasta

## Register the workflow

In [ ]:
import lamindb as ln
import json

How to amend a redun workflow.py to register input & output files in LaminDB?

- To register the workflow in the `Transform` registry, we added [the following lines](https://github.com/laminlabs/redun-lamin-fasta/blob/main/docs/workflow.py#L43-L49):

    ```python
    # register the workflow in the `Transform` registry
    transform = ln.Transform(
        name=redun_lamin_fasta.__name__,
        version=redun_lamin_fasta.__version__,
        type="pipeline",
        reference="https://github.com/laminlabs/redun-lamin-fasta",
    ).save()
    ```

- To query input files via LaminDB, we added [the following lines](https://github.com/laminlabs/redun-lamin-fasta/blob/main/docs/workflow.py#L50-L57):

    ```python
    # query & track this pipeline
    ln.track(transform=transform, params=params)
    # register input files in lamindb
    ln.save(ln.Artifact.from_dir(input_dir, run=False))
    # query input files from lamindb
    input_filepaths = [
        artifact.cache() for artifact in ln.Artifact.filter(key__startswith="fasta/")
    ]
    ```

- To register the output file via LaminDB, we added [the following line](https://github.com/laminlabs/redun-lamin-fasta/blob/main/redun_lamin_fasta/lib.py#L315):

    ```python
    ln.Artifact(output_path, description="results").save()
    ```

## Run redun

Let's see what the input files are:

In [ ]:
!ls ./fasta

And call the workflow:

In [ ]:
!redun run workflow.py main --input-dir ./fasta --tag run=test-run  1> redun_stdout.txt 2>redun_stderr.txt

Inspect the output:

In [ ]:
!cat redun_stdout.txt

And the error log:

In [ ]:
!tail -1 redun_stderr.txt

View data lineage:

In [ ]:
artifact = ln.Artifact.filter(description="results").one()  # query by description
artifact.view_lineage()

## Register the redun execution id

If we want to be able to query LaminDB for redun execution ID, this here is a way to get it:

In [ ]:
# export the run information from redun
!redun log --exec --exec-tag run=test-run --format json --no-pager > redun_exec.json
# load the redun execution id from the JSON and store it in the LaminDB run record
with open("redun_exec.json", "r") as file:
    redun_exec = json.loads(file.readline())
artifact.run.reference = redun_exec["id"]
artifact.run.reference_type = "redun_id"
artifact.run.save()

## Run report

Attach a run report:

In [ ]:
report = ln.Artifact(
    "redun_stderr.txt",
    description=f"Redun run report of {redun_exec['id']}",
    run=False,
    visibility=0,
).save()
artifact.run.report = report
artifact.run.save()
artifact.transform.latest_report
artifact.transform.save()

## View transforms and runs in LaminHub

https://lamin.ai/laminlabs/lamindata/transform/taasWKawCiNABiXZ

## View the database content

In [ ]:
ln.view()

Delete the test instance:

In [ ]:
!lamin delete --force redun-lamin-fasta